In [ ]:
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
from tqdm import tqdm

In [ ]:
def train_model_incrementally(file_path, existing_model, params=None, num_rounds=100):

       df1 = pd.read_csv(file_path)
       
       # Preprocess data
       df1.iloc[:, 2] = df1.iloc[:, 2].str.extract('(\d+)').astype(int)
       df1.iloc[:,-9:] = df1.iloc[:,-9:].applymap(lambda x: 1 if x != 0 else 0)
       df1 = df1.astype(int)   
       error_df = pd.DataFrame()
       fault_columns = [col for col in df1.columns if '故障' in col]
   
       for col in fault_columns:
           error_df[f'{col}_start'] = (df1[col] != 0) & (df1[col].shift(1) == 0)
           error_df[f'{col}_duration'] = df1.groupby((df1[col] == 0).cumsum())[col].transform('count') * error_df[f'{col}_start']
           duration_temp = pd.Series(index=error_df.index, dtype='float64')
           
           for i in error_df.index[error_df[f'{col}_start']]:
               duration_temp.iloc[i:i + error_df.at[i, f'{col}_duration']] = error_df.at[i, f'{col}_duration']
           
           error_df[f'{col}_duration'] = duration_temp.fillna(0).astype(int)
   
       for col in fault_columns:
           df1[f'{col}_duration'] = error_df[f'{col}_duration']
   
       X = df1.iloc[:, :-18].values
       y = df1.iloc[:, -18:-9].values
       
       X_tensor = torch.tensor(X, dtype=torch.float32)
       y_tensor = torch.tensor(y, dtype=torch.float32)
   
   #     oversampled_X_list = []
   #     oversampled_y_list = []
   
   #     for i in range(y.shape[1]):
   #         minority_class = y[:, i] == 1
   #         majority_class = ~minority_class
   #         target_minority_samples = int(X[majority_class].shape[0] / 3)
           
  #         X_minority_oversampled, y_minority_oversampled = resample(
   #             X[minority_class], y[minority_class][:, i],
  #             replace=True,
  #             n_samples=target_minority_samples,
   #             random_state=42
   #         )
           
   #         X_oversampled = np.vstack((X[majority_class], X_minority_oversampled))
   #         y_oversampled = np.hstack((y[majority_class][:, i], y_minority_oversampled))
          
  #         oversampled_X_list.append(X_oversampled)
   #         oversampled_y_list.append(y_oversampled)
   
   #     oversampled_X = np.concatenate(oversampled_X_list, axis=0)
   #     oversampled_y = np.concatenate(oversampled_y_list, axis=0)
       
       dtrain = xgb.DMatrix(X_tensor, label=y_tensor)
   
       # Set default parameters if none provided
       if params is None:
           params = {
               'max_depth': 3,
               'eta': 0.3,
               'objective': 'binary:logistic',
               'eval_metric': 'auc'
           }
   
       # Continue training the model
       bst = xgb.train(params, dtrain, num_rounds, xgb_model=existing_model)
   
       return bst
   #%% 
initial_model = None  # 如果有现有模型，将其路径或实例替换这里
trained_model = train_model_incrementally("F:\A\content\contenta\A.date\A题-全部数据 2\A题-全部数据\附件1\M101.csv", initial_model)

file_path_template = "F:\A\content\contenta\A.date\A题-全部数据 2\A题-全部数据\附件1\M{}.csv"
for i in tqdm(range(102,111)):
    filename = file_path_template.format(i)
    trained_model = train_model_incrementally(filename, trained_model)
   #%% 
   # 模型
model_path_json = "xgboost.json"  # 指定路径并使用 .json 扩展名
   
trained_model.save_model(model_path_json)

In [ ]:
import xgboost as xgb
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [ ]:
model_path_jsonn = './xgboost.json'

# 从 JSON 加载模型
loaded_bst_jsonn = xgb.Booster()
loaded_bst_jsonn.load_model(model_path_jsonn)
loaded_bst_jsonn

In [ ]:
def data_process(file_path):
    # Load data
    df1 = pd.read_csv(file_path)
    
    # Preprocess data
    df1.iloc[:, 2] = df1.iloc[:, 2].str.extract('(\d+)').astype(int)
    df1.iloc[:,-9:] = df1.iloc[:,-9:].applymap(lambda x: 1 if x != 0 else 0)
    df1 = df1.astype(int)

    error_df = pd.DataFrame()
    fault_columns = [colm for colm in df1.columns if '故障' in colm]

    for colm in fault_columns:
        error_df[f'{colm}_start'] = (df1[colm] != 0) & (df1[colm].shift(1) == 0)
        error_df[f'{colm}_duration'] = df1.groupby((df1[colm] == 0).cumsum())[colm].transform('count') * error_df[f'{colm}_start']
        duration_temp = pd.Series(index=error_df.index, dtype='float64')
        
        for i in error_df.index[error_df[f'{colm}_start']]:
            duration_temp.iloc[i:i + error_df.at[i, f'{colm}_duration']] = error_df.at[i, f'{colm}_duration']
        
        error_df[f'{colm}_duration'] = duration_temp.fillna(0).astype(int)

    for colm in fault_columns:
        df1[f'{colm}_duration'] = error_df[f'{colm}_duration']

    X = df1.iloc[:, :-18].values
    y = df1.iloc[:, -18:-9].values
    
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.float32)
    return X_tensor, y_tensor

In [ ]:
X, Y = data_process('F:\A\content\contenta\A.date\A题-全部数据 2\A题-全部数据\附件1/M101.csv')
X.shape, Y.shape

In [ ]:
# 在测试集上进行预测
dtest = xgb.DMatrix(X, label=Y)

preds = loaded_bst_jsonn.predict(dtest)
predictions = np.around(preds).astype(int)

# 计算准确率
accuracy = accuracy_score(Y, predictions)
print(f"Accuracy: {accuracy * 100.0}%")





In [ ]:
X, Y = data_process('F:\A\content\contenta\A.date\A题-全部数据 2\A题-全部数据\附件2\数据.csv')
X.shape, Y.shape

In [ ]:
# 在测试集上进行预测
dtest0 = xgb.DMatrix(X, label=Y)

preds0 = loaded_bst_jsonn.predict(dtest)
predictions = np.around(preds).astype(int)
# 假设outputs是你的神经网络输出的Tensor
# 转换为NumPy数组
outputs_np = predictions

# 绘制直方图
plt.hist(outputs_np, bins=9, alpha=0.8, label='Predicted Probabilities')
plt.xlabel('Predicted Probability')
plt.ylabel('Frequency')
plt.title('Distribution of Predicted Probabilities')
plt.legend()
plt.show()

# 或者绘制折线图
plt.plot(outputs_np, label='Predicted Probabilities')
plt.xlabel('Sample Index')
plt.ylabel('Predicted Probability')
plt.title('Predicted Probabilities for Each Sample')
plt.legend()
plt.show()

In [ ]:
pre = np.load('array.npy')
pre

In [ ]:
df = pd.read_csv('F:\A\content\contenta\A.date\A题-全部数据 2\A题-全部数据\附件2\数据.csv')
df

In [ ]:
# 将列表转换为 Pandas Series
series = pd.Series(pre[:,0])

# 计算故障开始的位置：1 出现的地方，并且前一个元素是 0
start = series[(series == 1) & (series.shift(1) != 1)].index

# 计算故障结束的位置：1 后面跟着 0 的位置
end = series[(series == 1) & (series.shift(-1) != 1)].index

# 计算每次故障的持续时间
durations = end - start + 1

# 创建一个 DataFrame 来保存故障的开始时间和持续时间
df1 = pd.DataFrame({
    'Fault Start': start,
    '持续时长/秒': durations
})

# 显示 DataFrame
df1 = df1[df1['持续时长/秒']>1]
df1.set_index('Fault Start', inplace=True)
result_df = df1.merge(df.iloc[:,:2], left_index=True, right_index=True, how='left')
df_reset = result_df.reset_index(drop=True)
df_reset['序号'] = df_reset.index
df_reset = df_reset.iloc[:,[3,1,2,0]]
df_reset

In [ ]:
dfs = []

# 遍历 prediction 的每一列
for i in range(pre.shape[1]):
    # 将列转换为 Pandas Series
    series = pd.Series(pre[:, i])
    
    # 计算故障开始的位置：1 出现的地方，并且前一个元素是 0
    start = series[(series == 1) & (series.shift(1) != 1)].index

    # 计算故障结束的位置：1 后面跟着 0 的位置
    end = series[(series == 1) & (series.shift(-1) != 1)].index

    # 计算每次故障的持续时间
    duration = end - start + 1

    # 创建一个 DataFrame 来保存故障的开始时间和持续时间
    df1 = pd.DataFrame({
        'Fault Start': start,
        '持续时长/秒': duration
    })

    # 显示 DataFrame
    df1 = df1[df1['持续时长/秒']>1]
    df1.set_index('Fault Start', inplace=True)
    df1 = df1.merge(df.iloc[:,:2], left_index=True, right_index=True, how='left')
    df1 = df1.reset_index(drop=True)
#     df1['序号'] = df1.index
    df1 = df1.iloc[:,[1,2,0]]
    
    # 添加到列表
    dfs.append(df1)

# 合并所有 DataFrame，横向并排
result_df = pd.concat(dfs, axis=1)

# 显示结果
result_df.to_csv('results2.csv')
result_df